In [1]:
from maxarseg.samplers import samplers
from maxarseg.geo_datasets import geoDatasets
from torch.utils.data import DataLoader
from torchgeo.datasets import stack_samples
from pathlib import Path
import os
import numpy as np
import matplotlib.pyplot as plt
from maxarseg.assemble import names

if Path.cwd().name != 'src':
    os.chdir('/nfs/home/vaschetti/maxarSrc/')

In [2]:
events_names = names.get_all_events()
for i, e_n in enumerate(events_names):
    print(i, e_n)

0 BayofBengal-Cyclone-Mocha-May-23
1 Emilia-Romagna-Italy-flooding-may23
2 Gambia-flooding-8-11-2022
3 Hurricane-Fiona-9-19-2022
4 Hurricane-Ian-9-26-2022
5 Hurricane-Idalia-Florida-Aug23
6 India-Floods-Oct-2023
7 Indonesia-Earthquake22
8 Kahramanmaras-turkey-earthquake-23
9 Kalehe-DRC-Flooding-5-8-23
10 Libya-Floods-Sept-2023
11 Marshall-Fire-21-Update
12 Maui-Hawaii-fires-Aug-23
13 Morocco-Earthquake-Sept-2023
14 NWT-Canada-Aug-23
15 New-Zealand-Flooding23
16 Sudan-flooding-8-22-2022
17 afghanistan-earthquake22
18 cyclone-emnati22
19 kentucky-flooding-7-29-2022
20 pakistan-flooding22
21 shovi-georgia-landslide-8Aug23
22 southafrica-flooding22
23 tonga-volcano21
24 yellowstone-flooding22


In [3]:
from maxarseg.assemble import holders

event = holders.Event(events_names[2],
                    seg_config = None,
                    det_config = None,
                    when='pre')
print(event.region_name)

all_mosaics_names = event.all_mosaics_names
print(all_mosaics_names)

m0 = event.mosaics[all_mosaics_names[2]]
print(f'Number of tiles: {m0.tiles_num}')

Creating event: Gambia-flooding-8-11-2022
Region: AfricaWest-Full
Mosaics: ['105001002BD68F00', '10300100CFC9A500', '1040010073D77D00']
AfricaWest-Full
['105001002BD68F00', '10300100CFC9A500', '1040010073D77D00']
Number of tiles: 10


In [4]:
m0.set_build_gdf()


Buildings: found 3 links matching: [33303200, 3330320, 333032]


In [5]:
from maxarseg.configs import SegmentConfig, DetectConfig
seg_config = SegmentConfig(batch_size = 1,
                        device='cpu',
                        size = 1024,
                        stride = 1024,
                        use_separate_detect_config = True,
                        clean_masks_bool=True,
                        ESAM_root='/nfs/home/vaschetti/maxarSrc/models/EfficientSAM'
                        )

det_config = DetectConfig(DF_batch_size = 1,
                        GD_batch_size= 1,
                        device='cpu',
                        size = 600,
                        stride = 600,
                        TEXT_PROMPT='bush',
                        BOX_THRESHOLD = 0.12,
                        DF_patch_size = 400,
                        DF_patch_overlap = 0.25,
                        min_ratio_GD_boxes_edges=0.5,
                        max_area_GD_boxes_mt2 = 6000,
                        nms_threshold=0.2
                        )

- Efficient SAM device: cpu


In [6]:
tile_path = m0.tiles_paths[1]
tile_path

PosixPath('/nfs/projects/overwatch/maxar-segmentation/maxar-open-data/Gambia-flooding-8-11-2022/pre/1040010073D77D00/033133031233.tif')

In [7]:
tile_path = m0.tiles_paths[1]

dataset = geoDatasets.MxrSingleTileNoEmpty(str(tile_path))
sampler = samplers.BatchGridGeoSampler(dataset = dataset,
                                    batch_size=seg_config.batch_size,
                                    size=seg_config.size,
                                    stride=seg_config.stride)

dataloader = DataLoader(dataset , batch_sampler=sampler, collate_fn=stack_samples)

In [8]:
iterabile = iter(dataloader)

In [11]:
from maxarseg.detect import detect

batch = next(iterabile)
sample_bbox = batch['bbox'][0]

In [12]:
sample_bbox

BoundingBox(minx=315156.25, maxx=315468.75, miny=1479843.75, maxy=1480156.25, mint=0.0, maxt=9.223372036854776e+18)

In [13]:
detect.evit_get_batch_box_prompt(sample_bbox, m0.proj_build_gdf, dataset.res)

tensor([[ 893.5878, 1021.0492,  940.8184, 1049.4922],
        [ 978.2892, 1013.2503, 1026.8911, 1043.5901],
        [ 895.2476,  984.5605,  940.4967, 1012.3614],
        [ 972.3121,  976.4455, 1022.8850, 1004.2693],
        [ 724.8817,  951.6296,  744.8765,  980.4776],
        [ 890.7768,  951.8585,  927.8968,  978.8636],
        [ 811.3453,  941.8502,  833.3995,  980.7298],
        [ 969.2638,  944.4745, 1017.7019,  972.5246],
        [ 938.3969,  938.9522,  956.3329,  976.1934],
        [ 986.2111,  904.4778, 1025.9043,  928.3371],
        [ 898.1642,  877.8641,  936.4798,  941.8895],
        [ 977.5417,  867.3012, 1029.6370,  893.0204],
        [ 879.4077,  814.8986,  904.4850,  855.3654],
        [ 918.9288,  812.2683,  944.0648,  853.6785],
        [ 960.1137,  810.3887,  983.8024,  851.5134],
        [ 998.7913,  805.9923, 1028.3145,  842.0572],
        [ 892.8147,  727.7225,  940.3717,  754.1353],
        [ 999.7088,  844.9842, 1032.2734,  860.2735],
        [1007.0009,  714.195

In [32]:
m0.name

'105001002BD68F00'